# Street Layout and Density

***17 September 2020***

Residential density appears to be a straighforward concept: some measure of housing (number of dwellings, building footprint, total floor area etc.) expressed as a ratio relative to land area. Increasing the size/volume of buildings as a mechanism for increasing density is well understood. Adding storeys or replacing small buildings with new ones with a bigger footprint are typical responses in the drive towards densification.

However, cities have a responsibility to manage the total area of land available to them. In simple terms this means the public spaces (including the highway) as well as the private land and the buildings built upon it. What is perhaps less well understood, particularly in cities that monitor density against plot or site area, is the role that the street network has to play in determining the relationship between urban form and overall density.

When 'optimising land use' (often a euphamism for increasing density) there is an interrelationship between the desired urban form, the street network and the resultant density. The relative permanence of the layout of the street network may explain the lack of focus on its role however, large scale urban regeneration projects and urban extensions are opportunities to revisit it.

This notebook is a first attempt at illustrating the relationship between the street network, urban form and density in two common types of urban form, the perimeter block and terrace/row housing. It demonstrates how two superficially similar building arrangements (abutting buildings aligned along the street) have fundamentally different relationships with the street network. 

Some key points to note:

- In both cases reducing block depth increases building density. Even though the area of developable land is reduced and the proportion of land given over to the street network is increased the tighter spacing between buildings (at the expense of garden space) increases gross density.
- In the case of the courtyard/perimeter block, reducing block length increases overall building density. Again a greater proportion of private land is transferred to public highway but again the quantity of building as a proportion of the whole increases. This is due the buildings making full use of the extra street frontage generated.
- In the case of row/terrace housing the reverse is true. Reducing block length decreases overall building density. In this case the extra frontage generated by giving land over to the public highway network is not compensated for by any extra building. The tighter street spacing appears to be a cost not only against the amount of private land but also the total quantity of building.
- Street width: in many so-called 'sustainable' developments there is pressure to increase the width of public highways to accommodate segregated cycle paths and to encourage walking. The width of the highway may also be increased to accommodate 'green' features such as trees and areas of grass. Again, particularly in systems which monitor density against plot area, it can go un-noticed that, if back gardens are not reduced to compensate, this will have the effect of driving down overall density which may be in direct conflict with the stated aims of the sustainable development. 
- The impact of set-back differs in the two typologies. In terrace/row housing it has no impact on overall density, it simply serves to move the the buildings forwards or backwards in their plots. In the courtyard/perimeter block typology, due to the uneven relationship between the front and back garden at corner plots, setting the building further back from the street decreases overall building density.

Increasing overall/gross density at a city scale is a simple matter of increasing the quantity of residential building relative to total land area. However it is worth remembering that at the city scale, the layout of the street network itself will impact both overall density and total capacity and that this may go unnoticed particularly if density is only monitored against the site or plot.

***References***

Berghauser Pont, M., & Haupt, P. (2009). Space, Density and Urban Form. https://doi.org/9789052693750

Martin, L., & March, L. (1972). Urban Space and Structures (L. Martin & L. March, eds.). London: Cambridge at the University Press.

Unwin, R. (1909). Town Planning in Practice: An Introduction to the Art of Designing Cities and Suburbs. London: T. Fisher Unwin.

## Imports

In [1]:
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, CustomJS, Slider
from bokeh.plotting import figure, output_file, show

In [2]:
# Configure the default output state to generate output saved to a file when show() is called.
output_file("Street_layout_and_density.html")

In [3]:
# Create a new Figure for plotting.
# A subclass of Plot that simplifies plot creation with default axes, grids, tools, etc.
# Figure objects have many glyph methods that can be used to draw vectorized graphical glyphs:

plot_l = figure(x_range=(0, 600),
                y_range=(0, 600),
                title="Courtyard/perimeter housing block"
                )
##plot_l.border_fill_color = "#eaeaf2"
plot_r = figure(x_range=(0, 600),
                y_range=(0, 600),
                title="Terrace/row housing block")
##plot_r.border_fill_color = "#eaeaf2"

```
x = [i for j in range(depth) for i in range(length)]
y = [j for j in range(depth) for i in range(length)]
```

In [4]:
num_wide = 12
num_deep = 12

block_length = 100
block_depth = 100

street_width = 12 # metres, includes pavement
set_back = 0 # metres, private front garden
building_depth = 10 # metres

# create Python street corners
bottom = [y*block_depth for y in range(num_deep) for x in range(num_wide)]
left = [x*block_length for y in range(num_deep) for x in range(num_wide)]
top = [bottom+block_depth for bottom in bottom]
right = [left+block_length for left in left]

# create Python land corners
land_bottom = [bottom + (street_width/2) for bottom in bottom]
land_left = [left + (street_width/2) for left in left]
land_top = [top - (street_width/2) for top in top]
land_right = [right - (street_width/2) for right in right]

# create Python land corners
courtyard_bottom = [land_bottom + building_depth for land_bottom in land_bottom]
courtyard_left = [land_left + building_depth for land_left in land_left]
courtyard_top = [land_top - building_depth for land_top in land_top]
courtyard_right = [land_right - building_depth for land_right in land_right]

# create row garden
row_garden_bottom = [land_bottom + building_depth for land_bottom in land_bottom]
row_garden_top = [land_top - building_depth for land_top in land_top]

Modeled on geoJSON - the data for the MultiPolygons glyph is different in that the vector of values is not a vector of scalars. Rather, it is a “list of lists of lists of lists”.

In [5]:
bottom_row_x_exteriors = [[list(exterior)] for exterior in zip(land_left, land_right, land_right, land_left)]
bottom_row_y_exteriors = [[list(exterior)] for exterior in zip(land_bottom, land_bottom, row_garden_bottom, row_garden_bottom)]

top_row_x_exteriors = [[list(exterior)] for exterior in zip(land_left, land_right, land_right, land_left)]
top_row_y_exteriors = [[list(interior)] for interior in zip(row_garden_top, row_garden_top, land_top, land_top)]

row_xs = [list(poly) for poly in zip(bottom_row_x_exteriors, top_row_x_exteriors)]
row_ys = [list(poly) for poly in zip(bottom_row_y_exteriors, top_row_y_exteriors)]

In [6]:
x_exteriors = [list(exterior) for exterior in zip(land_left, land_right, land_right, land_left)]
x_interiors = [list(interior) for interior in zip(courtyard_left, courtyard_left, courtyard_right, courtyard_right)]
y_exteriors = [list(interior) for interior in zip(land_bottom, land_bottom, land_top, land_top)]
y_interiors = [list(interior) for interior in zip(courtyard_bottom, courtyard_top, courtyard_top, courtyard_bottom)]

xs = [[list(poly)] for poly in zip(x_exteriors, x_interiors)]
ys = [[list(poly)] for poly in zip(y_exteriors, y_interiors)]

In [7]:
# Combine the Python lists into Bokeh's equivalent of a DataFrame
street_CDS = ColumnDataSource(data=dict(top=top,
                                        bottom=bottom,
                                        left=left,
                                        right=right))

# Combine the Python lists into Bokeh's equivalent of a DataFrame
land_CDS = ColumnDataSource(data=dict(land_bottom = land_bottom,
                                      land_top = land_top,
                                      land_left = land_left,
                                      land_right =land_right))

courtyard_CDS = ColumnDataSource(data=dict(xs = xs,
                                          ys = ys))

row_CDS = ColumnDataSource(data=dict(row_xs = row_xs,
                                     row_ys = row_ys))

In [8]:
# To draw axis aligned rectangles use the quad() glyph function specifying top, bottom, left and right positions:
plot_l.quad(left='left', right='right', top='top', bottom='bottom',
          source=street_CDS,
          line_color="white",
          line_dash="dashed",
          fill_color='#eee5dc',
          fill_alpha=1)
plot_r.quad(left='left', right='right', top='top', bottom='bottom',
          source=street_CDS,
          line_color="white",
          line_dash="dashed",
          fill_color='#eee5dc',
          fill_alpha=1)

GlyphRenderer(id='1079', ...)

In [9]:
# To draw axis aligned rectangles use the quad() glyph function specifying top, bottom, left and right positions:
plot_l.quad(left='land_left', right='land_right', top='land_top', bottom='land_bottom',
          source=land_CDS,
          fill_color='#e0dfdf', #'darkgrey'
          line_color='#b9b9b9',
          fill_alpha=1)
plot_r.quad(left='land_left', right='land_right', top='land_top', bottom='land_bottom',
          source=land_CDS,
          fill_color='#e0dfdf',
          line_color='#b9b9b9',
          fill_alpha=1)

GlyphRenderer(id='1091', ...)

Any Bokeh models that are configured in args (on the “Python side”) will automatically be available to the JavaScript code by the corresponding name. Additionally, the model that triggers the callback (i.e. the model that the callback is attached to) will be available as cb_obj.

In [10]:
plot_l.multi_polygons(xs='xs', ys='ys', source=courtyard_CDS, fill_color='#d9d0c9', line_color='#b6a99c')
plot_r.multi_polygons(xs='row_xs', ys='row_ys', source=row_CDS, fill_color='#d9d0c9', line_color='#b6a99c')

GlyphRenderer(id='1103', ...)

In [11]:
block_area = block_length * block_depth

private_area = (block_length - street_width) * (block_depth - street_width)

courtyard_building_area = (((block_length - street_width - 2*set_back) *
                            (block_depth - street_width - 2*set_back)) - 
                            ((block_length - street_width - 2*set_back - 2*building_depth) *
                             (block_depth - street_width - 2*set_back - 2*building_depth)))

row_building_area = (((block_length - street_width) *
                      (block_depth - street_width - 2*set_back)) - 
                     ((block_length - street_width) *
                      (block_depth - street_width - 2*set_back - 2*building_depth)))

street_proportion = 1 - (private_area/block_area)
courtyard_building_proportion = courtyard_building_area/block_area
row_building_proportion = row_building_area/block_area
courtyard_garden_proportion = (private_area - courtyard_building_area)/block_area
row_garden_proportion = (private_area - row_building_area)/block_area

In [12]:
block_types = ['Courtyard', 'Terrace']
land_types = ["Street", "Garden", "Building"]
fill_colors = ["#eee5dc", "#e0dfdf", "#d9d0c9"]
line_colors = ["#eee5dc", "#b9b9b9", "#b6a99c"]

land_use_data = {'block_types' : block_types,
                 'Street'   : [street_proportion, street_proportion],
                 'Garden'  : [courtyard_garden_proportion, row_garden_proportion],
                 'Building' : [courtyard_building_proportion, row_building_proportion]}

land_use_CDS = ColumnDataSource(data=land_use_data)

bar_plot = figure(x_range=block_types,
                  plot_height=350,
                  title="Land use split",
                  toolbar_location=None,
                  tools="")

bar_plot.vbar_stack(land_types, #v
                    x='block_types', #x
                    width=0.4,
                    fill_color=fill_colors,
                    line_color=line_colors,
                    source=land_use_CDS,
                    legend_label=land_types)

bar_plot.y_range.start = 0
bar_plot.x_range.range_padding = 0.1
bar_plot.xgrid.grid_line_color = None
bar_plot.axis.minor_tick_line_color = None
bar_plot.outline_line_color = None
bar_plot.legend.location = "top_right"
bar_plot.legend.orientation = "vertical"
##bar_plot.border_fill_color = "#eaeaf2"

In [13]:
# instantiate Sliders
block_length_slider = Slider(start=building_depth * 2, end=600, value=100, step=1, title="Block length (m)")
block_depth_slider = Slider(start=building_depth * 2, end=300, value=100, step=1, title="Block depth (m)")
street_width_slider = Slider(start=2, end=50, value=street_width, step=0.5, title="Street width (m)")
building_depth_slider = Slider(start=5, end=20, value=building_depth, step=0.5, title="Building depth (m)")
set_back_slider = Slider(start=0, end=200, value=0, step=0.5, title="Set back or front garden (m)")
storeys_slider = Slider(start=0, end=100, value=2, step=1, title="Number of storeys")

## Create the callback
***Note: any models passed in as arguments must already be defined***

In [14]:
# ALTERNATIVE: PASSING NAME OF POWER SLIDER, NOT RELYING ON cb_obj.value

callback = CustomJS(args = dict(street_CDS_source = street_CDS,
                                land_CDS_source = land_CDS,
                                courtyard_CDS_source = courtyard_CDS,
                                row_CDS_source = row_CDS,
                                land_use_CDS_source = land_use_CDS,
                                block_length_slider = block_length_slider,
                                block_depth_slider = block_depth_slider,
                                street_width_slider = street_width_slider,
                                building_depth_slider = building_depth_slider,
                                set_back_slider = set_back_slider,
                                storeys_slider = storeys_slider,
                               ),
                    code="""
    
        // access the data in the ColumnDataSource through a variable called `data`
        var street_data = street_CDS_source.data;
        var land_data = land_CDS_source.data;
        var courtyard_data = courtyard_CDS_source.data;
        var row_data = row_CDS_source.data;
        var land_use_data = land_use_CDS_source.data;
        
        // get the slider values:
        var block_length = block_length_slider.value;
        var block_depth = block_depth_slider.value;
        var street_width = street_width_slider.value;
        var building_depth = building_depth_slider.value;
        var set_back = set_back_slider.value;
        var storeys = storeys_slider.value;
        
        // bind values in the ColumnDataSource to variables
        var street_top = street_data['top'];
        var street_bottom = street_data['bottom'];
        var street_left = street_data['left'];
        var street_right = street_data['right'];
        
        var land_top = land_data['land_top'];
        var land_bottom = land_data['land_bottom'];
        var land_left = land_data['land_left'];
        var land_right = land_data['land_right'];
        
        var courtyard_xs = courtyard_data['xs'];
        var courtyard_ys = courtyard_data['ys'];
        
        var row_xs = row_data['row_xs'];
        var row_ys = row_data['row_ys'];
        
        var land_use_street = land_use_data['Street'];
        var land_use_garden = land_use_data['Garden'];
        var land_use_building = land_use_data['Building'];
        
        // using the length of the x values to control the loop
        var num_wide = 12;
        var num_deep = 12;

        var k = 0;

        for (var y = 0; y < num_deep; y++) {
            for (var x = 0; x < num_wide; x++) {

                // update the values
                street_bottom[k] = y * block_depth;
                street_top[k] = street_bottom[k] + block_depth;
                street_left[k] = x * block_length;
                street_right[k] = street_left[k] + block_length;

                land_bottom[k] = street_bottom[k] + street_width/2;
                land_top[k] = street_top[k] - street_width/2;
                land_left[k] = street_left[k] + street_width/2;
                land_right[k] = street_right[k] - street_width/2;
                
                courtyard_xs[k][0][0][0] = land_left[k] + set_back
                courtyard_xs[k][0][0][1] = land_right[k] - set_back
                courtyard_xs[k][0][0][2] = land_right[k] - set_back
                courtyard_xs[k][0][0][3] = land_left[k] + set_back
                courtyard_xs[k][0][1][0] = land_left[k] + set_back + building_depth
                courtyard_xs[k][0][1][1] = land_left[k] + set_back + building_depth
                courtyard_xs[k][0][1][2] = land_right[k] - set_back - building_depth
                courtyard_xs[k][0][1][3] = land_right[k] - set_back - building_depth
                
                courtyard_ys[k][0][0][0] = land_bottom[k] + set_back
                courtyard_ys[k][0][0][1] = land_bottom[k] + set_back
                courtyard_ys[k][0][0][2] = land_top[k] - set_back
                courtyard_ys[k][0][0][3] = land_top[k] - set_back
                courtyard_ys[k][0][1][0] = land_bottom[k] + set_back + building_depth
                courtyard_ys[k][0][1][1] = land_top[k] - set_back - building_depth
                courtyard_ys[k][0][1][2] = land_top[k] - set_back - building_depth
                courtyard_ys[k][0][1][3] = land_bottom[k] + set_back + building_depth

                row_xs[k][0][0][0] = land_left[k]
                row_xs[k][0][0][1] = land_right[k]
                row_xs[k][0][0][2] = land_right[k]
                row_xs[k][0][0][3] = land_left[k]
                row_xs[k][1][0][0] = land_left[k]
                row_xs[k][1][0][1] = land_left[k]
                row_xs[k][1][0][2] = land_right[k]
                row_xs[k][1][0][3] = land_right[k]
                
                row_ys[k][0][0][0] = land_bottom[k] + set_back
                row_ys[k][0][0][1] = land_bottom[k] + set_back
                row_ys[k][0][0][2] = land_top[k] - set_back
                row_ys[k][0][0][3] = land_top[k] - set_back
                row_ys[k][1][0][0] = land_bottom[k] + set_back + building_depth
                row_ys[k][1][0][1] = land_top[k] - set_back - building_depth
                row_ys[k][1][0][2] = land_top[k] - set_back - building_depth
                row_ys[k][1][0][3] = land_bottom[k] + set_back + building_depth
                
                k++;
            }
        }
        
        // Update the bar chart
        var block_area = block_length * block_depth;

        var private_area = (block_length - street_width) * (block_depth - street_width);

        var courtyard_building_area = (((block_length - street_width - 2*set_back) *
                                        (block_depth - street_width - 2*set_back)) - 
                                       ((block_length - street_width - 2*set_back - 2*building_depth) *
                                        (block_depth - street_width - 2*set_back - 2*building_depth)));

        var row_building_area = (((block_length - street_width) *
                                  (block_depth - street_width - 2*set_back)) - 
                                 ((block_length - street_width) *
                                  (block_depth - street_width - 2*set_back - 2*building_depth)));

        var street_proportion = 1 - (private_area/block_area);
        var courtyard_building_proportion = courtyard_building_area/block_area;
        var row_building_proportion = row_building_area/block_area;
        var courtyard_garden_proportion = (private_area - courtyard_building_area)/block_area;
        var row_garden_proportion = (private_area - row_building_area)/block_area;
        
        land_use_street[0] = street_proportion;
        land_use_street[1] = street_proportion;
        land_use_garden[0] = courtyard_garden_proportion;
        land_use_garden[1] = row_garden_proportion;
        land_use_building[0] = courtyard_building_proportion;
        land_use_building[1] = row_building_proportion;
        
        // update the ColumnDataSource with the new values
        street_CDS_source.change.emit();
        land_CDS_source.change.emit();
        courtyard_CDS_source.change.emit();
        row_CDS_source.change.emit();
        land_use_CDS_source.change.emit();
    """)

These CustomJS callbacks can be attached to property change events on any Bokeh model, using the js_on_change method of Bokeh models:

`p = figure()`

`# execute a callback whenever p.x_range.start changes`

`p.x_range.js_on_change('start', callback)`

***Note here how the js_on_change event is triggered by the x_range property of p, not p itself***

In [15]:
# 
block_length_slider.js_on_change('value', callback)
block_depth_slider.js_on_change('value', callback)
street_width_slider.js_on_change('value', callback)
building_depth_slider.js_on_change('value', callback)
set_back_slider.js_on_change('value', callback)
storeys_slider.js_on_change('value', callback)

In [16]:
# create the layout
layout = column(row(plot_l,
                    plot_r,
                    column(bar_plot,
                           block_length_slider,
                           block_depth_slider,
                           street_width_slider,
                           building_depth_slider,
                           set_back_slider)))#,
                           #storeys_slider)))

# and show it (and output it to the file specified at the beginning)
show(layout)